In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [6]:
!pip install swig
!pip install box2d-py
!pip install gymnasium
!pip install gymnasium[box2d]
!pip install stable-baselines3[extra]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.5/374.5 kB 21.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for box2d-py: filename=box2d_py-2.3.8-cp310-cp310-linux_x86_64.whl size=2376104 sha256=0c2f4ba3d61f6579a6540bd48bab8c86b0dcc168d96fa856ac435ade6ae2adcf
  Stored in directory: /root/.cache/pip/wheels/47/01/d2/6a780da77ccb98b1d2facdd520a8d10838a03b590f6f8d50c0
Successfully built box2d-py
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 21.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for box2d-py: filename=box2d_py-2.3.5-cp310-cp310-linux_x86_64.whl size=2376133 sha256=daa929714da43ce06e8afa888f686868d5e19015b7c3a33b67e0da391d4e230b
  Stored in directory: /root/.cache/pip/wheels/db/8f/6a/eaaadf056fba10a98d986f6dce954e6201ba3126926fc5ad9e
Successfully bui

Installations to set up virtual display to render env in colab

In [7]:
!apt-get update > /dev/null 2>&1
!apt-get install -y xvfb > /dev/null 2>&1

In [8]:
!pip install pyvirtualdisplay > /dev/null 2>&1
!pip install gym==0.26.0 gym[box2d] pyglet==1.5.27 > /dev/null 2>&1

We are setting up functions to be able to render the environment within the cell output through the code below

In [9]:
import gymnasium as gym
from gymnasium.wrappers import RecordVideo
import glob
import io
import base64
from IPython.display import HTML
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay

# Start virtual display
display = Display(visible=0, size=(1400, 900))
display.start()

"""
Utility functions to enable video recording of gymnasium environment
and displaying it.
To enable video, just do "env = wrap_env(env)"
"""

def show_video():
    mp4list = glob.glob('videos/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")


def wrap_env(env):
    # env = RecordVideo(env, video_folder='videos', episode_trigger=lambda episode_id: True)
    # return env
    base_video_folder = "videos"
    video_folder = base_video_folder
    # Create a new folder name if it already exists
    i = 0
    while os.path.exists(video_folder):
        i += 1
        video_folder = f"{base_video_folder}_{i}"
    # Wrap environment with the new unique video folder
    env = RecordVideo(env, video_folder=video_folder, episode_trigger=lambda episode_id: True)
    return env

In [10]:
import os
import gymnasium as gym
from stable_baselines3 import PPO #Proximal policy optimization
from stable_baselines3.common.vec_env import DummyVecEnv #to vectorize the environment
from stable_baselines3.common.evaluation import evaluate_policy #returns average rewards for episodes and also std deviation for our agent

### TESTING THE ENVIORNMENT

In [11]:
env_name='CarRacing-v2'
Sample_env = wrap_env(gym.make("CarRacing-v2", render_mode="rgb_array"))

In [12]:
print(f"Observation space: {Sample_env.observation_space}")
print(f"Action space: {Sample_env.action_space}")

Observation space: Box(0, 255, (96, 96, 3), uint8)
Action space: Box([-1.  0.  0.], 1.0, (3,), float32)


Rendering the envrironment and visualizing agent (car) in the race course

In [13]:
observation, info = Sample_env.reset()
while True:
    action = Sample_env.action_space.sample()
    observation, reward, done, truncated, info = Sample_env.step(action)
    if done or truncated:
        break
Sample_env.close()
show_video()

Moviepy - Building video /content/videos/rl-video-episode-0.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-0.mp4


### Training the agent

In [14]:
env=gym.make(env_name)
env=DummyVecEnv([lambda:env])

In [15]:
log_path=os.path.join('Training','Logs')
car_ppo=PPO('CnnPolicy',env,verbose=1,tensorboard_log=log_path)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [16]:
total_timesteps=100000

In [17]:
car_ppo.learn(total_timesteps)

Logging to Training/Logs/PPO_1
-----------------------------
| time/              |      |
|    fps             | 71   |
|    iterations      | 1    |
|    time_elapsed    | 28   |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 68           |
|    iterations           | 2            |
|    time_elapsed         | 60           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0029231776 |
|    clip_fraction        | 0.0258       |
|    clip_range           | 0.2          |
|    entropy_loss         | -4.25        |
|    explained_variance   | -0.025       |
|    learning_rate        | 0.0003       |
|    loss                 | 0.161        |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.00257     |
|    std                  | 0.997        |
|    value_loss           | 

In [18]:
ppo_path=os.path.join('Training','Saved Models','PPO_Driving_Car')
car_ppo.save(ppo_path)

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/save_util.py:284: UserWarning: Path 'Training/Saved Models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [19]:
# ppo_model=PPO.load(ppo_path,env)